# Highway corridors

See which highway corridors have a lot of parallel routes per mile, but few competitive routes per mile.

Draw quadrant?

Issue: highway-route pairs mean that route might only be parallel to 1 highway segment and not the other highways it intersects with. taking the `max` when grouping by highway segment...which then throws off competitive and parallel...which now can be different

In [1]:
import geopandas as gpd
import intake
import pandas as pd

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH

catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0430 21:51:50.609380383    1024 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0430 21:51:52.884592402    1024 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()

In [3]:
from shared_utils import geography_utils
# parallel routes / competitive routes per mile
# or percents?
# right now, highway_length is in feet
hwy_stats = hwy_stats.assign(
    parallel_per_mi = (hwy_stats.num_parallel.divide(hwy_stats.highway_length) * 
                       geography_utils.FEET_PER_MI),
    competitive_per_mi = (hwy_stats.num_competitive.divide(hwy_stats.highway_length) * 
                          geography_utils.FEET_PER_MI),
)

In [4]:
# scatterplot
import altair as alt
from shared_utils import styleguide

alt.themes.register("calitp_theme", styleguide.calitp_theme)

<function shared_utils.styleguide.calitp_theme(font='Raleway', labelFont='Nunito Sans', font_size=18, chart_width=400, chart_height=250, markColor='#8CBCCB', axisColor='#cbcbcb', guideLabelColor='#474747', guideTitleColor='#333', blackTitle='#333', backgroundColor='white', PALETTE={'category_bright': ['#2EA8CE', '#EB9F3C', '#F4D837', '#51BF9D', '#8CBCCB', '#9487C0'], 'category_bold': ['#136C97', '#E16B26', '#F6BF16', '#00896B', '#7790A3', '#5B559C'], 'diverging': ['#E16B26', '#EB9F3C', '#f6e7e1', '#8CBCCB', '#2EA8CE', '#136C97'], 'sequential': ['#B9D6DF', '#8CBCCB', '#2EA8CE', '#136C97', '#0B405B']})>

In [5]:
hwy_stats[hwy_stats.pct_parallel==0].shape

(282, 13)

In [34]:
def labeling(word):
    rename_dict = {
        "pct_parallel": "% parallel routes",
        "pct_competitive": "% competitive routes"
    }
    
    if word in rename_dict.keys():
        word = rename_dict[word]
    else:
        word = word.replace('_', ' ').title()
        
    return word

def make_scatterplot(df, x_col, y_col):
    
    if "pct" in x_col:
        x_p50 = 0.5
        y_p50 = 0.5
    else:
        x_p50 = 0.1
        y_p50 = 0.1
    
    chart = (alt.Chart(df)
             .mark_point(size=50)
             .encode(
                 x=alt.X(f"{x_col}:Q", title=labeling(x_col)),
                 y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
                 color=alt.Color("District:N"),
                 tooltip=["Route", "County", "District",
                          "num_parallel", "pct_parallel", "parallel_per_mi",
                          "num_competitive", "pct_competitive", "competitive_per_mi",
                 ]
             ).interactive()
             .properties(title = f"{labeling(x_col)} vs {labeling(y_col)}")
    )
    
    horiz_line = (alt.Chart(df.assign(y_p50=y_p50))
                  .mark_rule(strokeDash=[2,3])
                  .encode(
                      y="y_p50:Q",
                      color=alt.value("black")
                  )
    )
    
    vertical_line = (alt.Chart(df.assign(x_p50=x_p50))
                     .mark_rule(strokeDash=[2,3])
                     .encode(
                         x="x_p50:Q",
                         color=alt.value("black")
                     )
    )
    
    combined = chart + horiz_line + vertical_line
    return combined


In [37]:
make_scatterplot(hwy_stats[hwy_stats.pct_parallel > 0], "pct_parallel", "pct_competitive")

YES


alt.LayerChart(...)

In [38]:
make_scatterplot(hwy_stats[hwy_stats.pct_parallel > 0], "parallel_per_mi", "competitive_per_mi")

NO


alt.LayerChart(...)